# **Prédiction sur data_test en utilisant les éléments précédents**

Je vais détailler ici toutes les étapes à la main, mais si l'on souhaitait mettre cela en production il faudrait commencer par ranger tous ces éléments dans une fonction.

In [14]:
import pandas as pd
import numpy as np
import pickle
import joblib
import func_custom as fc

# 1. Importation des données et pièces du modèle

In [2]:
# Données à prédire, je laisse "Unnamed: 0" car c'est probablement via ce numéro que vous ferez l'évaluation du classifieur
df_test = pd.read_excel("data/data_test.xlsx")

In [3]:
# Stopwords issus de 1. Preprocessing
with open('data/stopwords_complete.pkl', 'rb') as file:
    stopwords_complete = pickle.load(file)

# Vectorizer de TF_IDF
with open('predictor/vectorizer_tfidf.pkl', 'rb') as file:
    vectorizer = pickle.load(file)

# Modèles TF_IDF avec hyperparamètres optimisés
best_model_1 = joblib.load("predictor/tfidf_label1.joblib")
best_model_8 = joblib.load("predictor/tfidf_label8.joblib")

# 2. Application du preprocessing, vectoriser, predictor

In [4]:
df_test["message_clean"] = df_test["message"].apply(lambda x : fc.preprocess_text(x, stopwords_complete))

c:\Users\TitouanBlaize\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\TitouanBlaize\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [5]:
df_test

,Unnamed: 0,message,message_clean
0,0,"Chère Madame, Cher Monsieur, Impossible de ch...",cher cher impossible changer situation familia...
1,1,"Bonjour, mon taux pour le prélévement à la ...",taux prélévement source évaluer percevoir reta...
2,2,"Bonsoir, Le 0000.0000.0000 ma première fille ...",bonsoir premier fille naître ensuite pacser so...
3,3,"Bonjour, Lors de ma déclaration de changement...",lors déclaration changement situation pacs nai...
4,4,Bonjour concernant le prélèvement a la sourc...,concerner prélèvement source souhaiter départ ...
...,...,...,...
295,295,"Bonjour, j'ai effectué ma déclaration dernie...",effectuer déclaration dernièrement aimer savoi...
296,296,Bonjour je voulais faire la mise à jour de ma...,vouloir mise jour situation fiscal déclarer pa...
297,297,"Bonjour, j’ai déclaré ce jour la conclusion ...",déclarer jour conclusion pacs conjoint étonner...
298,298,"Bonjour, Je viens de déclarer un changement d...",venir déclarer changement situation familial c...


In [6]:
# Surtout SURTOUT juste transform et pas fit_transform car on est en test, il ne faut pas de data leakage
X_test = vectorizer.transform(df_test["message_clean"])

In [7]:
df_test["label_1"] = best_model_1.predict(X_test)
df_test["label_8"] = best_model_8.predict(X_test)

# 3. Label final et sauvegarde du résultat

In [8]:
# Combinaison simple des deux lables prédits respectivement
def multilabel(row):
    if row["label_1"]:
        if row["label_8"]:
            return "1:8"
        else:
            return "1"
    else:
        return "8"

In [9]:
df_test["label"] = df_test.apply(multilabel, axis = 1)

In [10]:
df_test["label"].value_counts()

label
8      137
1      120
1:8     43
Name: count, dtype: int64

In [17]:
print(f"Si on suppose que l'échantillon test est similaire à celui de train (Label 8 : 309, Label 1 : 302, Label 1:8 : 89) on retombe sur des proportions similaires ce qui est rassurant. Par exemple le rapport 1:8/8 était de {np.round(100*89/309, 1)} contre {np.round(100*43/137, 1)} dans le test")

Si on suppose que l'échantillon test est similaire à celui de train (Label 8 : 309, Label 1 : 302, Label 1:8 : 89) on retombe sur des proportions similaires ce qui est rassurant. Par exemple le rapport 1:8/8 était de 28.8 contre 31.4 dans le test


In [11]:
# Pour garder trace des étapes
df_test.to_csv("df_test_predict_full.csv", sep = ";", index = False)

# Pour vérification uniquement
df_test[["message", "label"]].to_csv("df_test_predict.csv", sep = ";", index = False)

# 4. Pour aller plus loin

Voici quelques idées supplémentaires à investiguer dans l'espoir d'améliorer les résultats :
- Travail encore plus fin pour bien éliminer les stopwords et analyse approndie des erreurs de tokenizer/lemmetizer
- Utilisation d'autres LLM que Camembert, notamment si corpus plus adaptés à "l'oralité" des messages + tokens en input plus longs que 512
- Les deux classes de modèles les plus performantes semblent ici être le simple TF-IDF ou les LLM Français, on pourrait analyser les messages classer différement de l'un à l'autre pour mieux comprendre les différences
- On pourrait même mettre en place un modèle ensembliste avec vote par modèle, et analyser plus finement lorsque les modèles votent différemment
- Dans le même ordre idée, inspecter les erreurs le long de la distribution des scores (lorsque le score est très proche de 0 ou 1 est-ce qu'on se trompe encore ?) et composer différement les modèles alors